<a href="https://colab.research.google.com/github/Vik-Gt-321/Tweet-Sentiment-Analysis/blob/main/Sentiment_Analysis_Of_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import random

Things to try out:

Use pretrained embeddings

Use one_hot

https://jalammar.github.io/illustrated-word2vec/

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/NLP/Corona_NLP_train.csv.zip")
zip_ref.extractall()
zip_ref.close

<bound method ZipFile.close of <zipfile.ZipFile filename='/content/drive/MyDrive/NLP/Corona_NLP_train.csv.zip' mode='r'>>

In [ ]:
train_df = pd.read_csv("/content/Corona_NLP_train.csv", engine="python")

In [ ]:
train_df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
test_df = pd.read_csv("/content/Corona_NLP_test.csv", engine="python")

In [ ]:
train_df_shuffled = train_df.sample(frac =1, random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_df_shuffled["Sentiment"] = label_encoder.fit_transform(train_df_shuffled.Sentiment)

In [ ]:
train_df_shuffled.Sentiment.value_counts()

4    11422
2     9917
3     7713
1     6624
0     5481
Name: Sentiment, dtype: int64

In [ ]:
len(train_df), len(test_df)

(41157, 3798)

## Visualize Random Texts

In [ ]:
random_index = random.randint(0, len(train_df_shuffled)-5)
for row in train_df_shuffled[random_index:random_index+5].itertuples():
  _,  _, _, _, _, tweet, sentiment = row
  print(f"The text: \n {tweet}")
  print(f"Sentiment: {sentiment}")
  print("-----")

The text: 

https://t.co/AMA9MtXxKy
Sentiment: 0
-----
The text: 

@Datassential #coronavirusrestaurants #restaurantnews https://t.co/LpNPnKcKKx
Sentiment: 0
-----
The text: 

How can I help..? Can I buy food bring to nurses ? I heard that,one of nurse after she's been working 48h nonstop....??when she went to buy something to eat in the supermarket it was no food left ?? this make me so sad ?#ukgovernment #coronavirus @BorisJohnson
Sentiment: 2
-----
The text: 
 warning at 32c this morning in Please don t panic buy goods Please to a food bank like if able We re all
Sentiment: 4
-----
The text: 



Account Name: LAGOS FOODhttps://t.co/XrwKy2FTi4
Sentiment: 1
-----


## Data manipulation

In [ ]:
from sklearn.model_selection import train_test_split
train_sent, val_sent, train_labels, val_labels =train_test_split(train_df_shuffled["OriginalTweet"].to_numpy(), 
                                                                 train_df_shuffled["Sentiment"].to_numpy(), 
                                                                 test_size = 0.2, 
                                                                 random_state = 42)

In [ ]:
len(train_sent), len(val_sent)

(32925, 8232)

In [ ]:
len(train_labels), len(val_labels)

(32925, 8232)

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization 
text_vectorizer = TextVectorization()

In [ ]:
train_sent[:2]

array(['Incidentally assistant on the checkout this morning said shoppers being more sensible now whereas last couple of weeks they went bloody mad they did',
       '#coronavirus #scams: What the #FTC is doing \r\r\n#COVID #infosec #CoronaVirusUpdates #Covid_19 #COVID19\r\r\n\r\r\nhttps://t.co/lSFnxmUnpb'],
      dtype=object)

In [ ]:
arr = [len(i.split()) for i in train_sent]
round(np.sum(arr)/len(arr))

31

In [ ]:
max_vocab_length = 10000
max_len = 31
text_vectorizer = TextVectorization(max_tokens=max_vocab_length, 
                                    output_sequence_length =max_len)

In [ ]:
text_vectorizer.adapt(train_sent)

In [ ]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 31), dtype=int64, numpy=
array([[ 667,    6, 4258,    7,   38,  926,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]])>

In [ ]:
rand_sent = random.choice(train_sent)
rand_sent, len(rand_sent)

('It would be wonderful if as you prepare your family and stock up folks could refrain from panic buying and hoarding extras This goes a long way to ensure there is enough food for everyone',
 187)

In [ ]:
text_vectorizer([rand_sent])

<tf.Tensor: shape=(1, 31), dtype=int64, numpy=
array([[  29,  141,   28, 2546,   50,   27,   14, 1222,   34,  278,    4,
          85,   43,  733,  144, 4781,   31,   69,   95,    4,  292, 8559,
          17,  746,    6,  252,  172,    3,  555,   82,   11]])>

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab[:5], words_in_vocab[-5:]

(['', '[UNK]', 'the', 'to', 'and'],
 ['adexchanger', 'adaptation', 'acquiring', 'acknowledge', 'acid'])

In [ ]:
len(words_in_vocab)

10000

In [ ]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = len(words_in_vocab), 
                             output_dim =128, 
                             input_length = max_len, 
                             embeddings_initializer = "uniform")

In [ ]:
embedding(text_vectorizer([rand_sent]))

<tf.Tensor: shape=(1, 31, 128), dtype=float32, numpy=
array([[[-4.6221580e-02,  4.1820016e-02, -4.6838153e-02, ...,
          2.5758255e-02,  1.6018692e-02,  5.9864633e-03],
        [-1.9077683e-02, -2.6180958e-02, -2.5845839e-02, ...,
         -2.1403586e-02, -4.6151865e-02,  3.4272578e-02],
        [ 1.0225333e-02,  2.9215422e-02, -1.3597906e-02, ...,
          4.9495701e-02,  4.2785693e-02, -1.3602007e-02],
        ...,
        [ 2.9138137e-02, -9.5486641e-05,  1.4468860e-02, ...,
         -4.2608883e-02, -4.4569243e-02,  3.6848560e-03],
        [ 1.0552466e-02,  3.0727498e-03, -2.3840740e-04, ...,
          1.5409138e-02, -4.5953371e-02,  1.3841763e-03],
        [-3.0095173e-02, -6.3285343e-03, -3.8970865e-02, ...,
          4.7807481e-02, -4.7387958e-02, -4.4168781e-02]]], dtype=float32)>

## Models

In [ ]:
import datetime
def create_tensorboard_callback(dirname, exp_name):
  logdir = dirname + "/" + exp_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tens_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
  return tens_callback
SAVE_DIR = "model_logs"

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
model_0 = Pipeline([
                    ("tfidf",  TfidfVectorizer()), 
                    ("clf",  MultinomialNB())
])
model_0.fit(train_sent, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
baseline_score = model_0.score(val_sent, val_labels)

AttributeError: ignored

In [ ]:
baseline_score

0.3554421768707483

In [ ]:
basline_preds = model_0.predict(val_sent)
basline_preds[:5]

array([4, 4, 4, 4, 2])

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
calculate_results(val_labels, basline_preds)

{'accuracy': 35.544217687074834,
 'f1': 0.250356137992476,
 'precision': 0.6264372191583794,
 'recall': 0.3554421768707483}

## Model 1 

In [ ]:
inputs = layers.Input(shape  =(1,), dtype = "string")
x  = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(5, activation="softmax")(x)
model_1 = tf.keras.Model(inputs, outputs)
model_1.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_9 (TextVe (None, 31)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 31, 128)           1280000   
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 645       
Total params: 1,280,645
Trainable params: 1,280,645
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_1.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_1.fit(train_sent, train_labels, 
            validation_data = (val_sent, val_labels), 
            epochs = 5, 
            callbacks = [create_tensorboard_callback(SAVE_DIR, "model_1_dense")])

Epoch 1/5
1029/1029 [==============================] - 19s 18ms/step - loss: 1.3383 - accuracy: 0.4259 - val_loss: 1.1127 - val_accuracy: 0.5581
Epoch 2/5
1029/1029 [==============================] - 17s 17ms/step - loss: 0.9720 - accuracy: 0.6334 - val_loss: 0.9629 - val_accuracy: 0.6382
Epoch 3/5
1029/1029 [==============================] - 17s 17ms/step - loss: 0.7772 - accuracy: 0.7362 - val_loss: 0.9199 - val_accuracy: 0.6589
Epoch 4/5
1029/1029 [==============================] - 17s 17ms/step - loss: 0.6256 - accuracy: 0.8046 - val_loss: 0.9236 - val_accuracy: 0.6529
Epoch 5/5
1029/1029 [==============================] - 17s 17ms/step - loss: 0.4946 - accuracy: 0.8584 - val_loss: 0.9536 - val_accuracy: 0.6427


In [ ]:
model_1_predprobs = model_1.predict(val_sent)
model_1_predprobs[:3]

array([[5.8511055e-01, 1.6451550e-03, 2.8116843e-01, 9.5223583e-02,
        3.6852352e-02],
       [5.2329700e-04, 6.5941167e-01, 7.7162101e-03, 1.3480951e-02,
        3.1886783e-01],
       [5.8211561e-02, 1.6475031e-03, 4.3371624e-01, 2.3488341e-01,
        2.7154133e-01]], dtype=float32)

In [ ]:
val_labels[:3]

array([2, 1, 2])

In [ ]:
model_1_preds = tf.argmax(model_1_predprobs, axis =1)

In [ ]:
model_1_preds

<tf.Tensor: shape=(8232,), dtype=int64, numpy=array([0, 1, 2, ..., 2, 0, 2])>

In [ ]:
calculate_results(val_labels, model_1_preds)

{'accuracy': 64.27356656948494,
 'f1': 0.6423454524708176,
 'precision': 0.6448364145430189,
 'recall': 0.6427356656948494}

In [ ]:
def calc_results_from_model(model):
  model_pred_probs = model.predict(val_sent)
  model_preds = tf.argmax(model_pred_probs, axis =1)
  return calculate_results(model_preds, val_labels)

In [ ]:
model_1_results = calc_results_from_model(model_1)

In [ ]:
model_1_results

{'accuracy': 64.27356656948494,
 'f1': 0.6431258789188811,
 'precision': 0.6461964278553304,
 'recall': 0.6427356656948494}

In [ ]:
# can try vector visualization too

## LSTM model

In [ ]:
inputs = layers.Input(shape =(1, ), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64)(x) # change return seq to true when stacking layers
outputs = layers.Dense(5, activation="softmax")(x)
model_2  = tf.keras.Model(inputs, outputs)
model_2.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_9 (TextVe (None, 31)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 31, 128)           1280000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 325       
Total params: 1,329,733
Trainable params: 1,329,733
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_2.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_2_history = model_2.fit(train_sent, train_labels, 
                              validation_data = (val_sent, val_labels), epochs =5,
                              callbacks = [create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")])

Epoch 1/5
1029/1029 [==============================] - 44s 41ms/step - loss: 0.9781 - accuracy: 0.6027 - val_loss: 0.8670 - val_accuracy: 0.6691
Epoch 2/5
1029/1029 [==============================] - 40s 39ms/step - loss: 0.6752 - accuracy: 0.7559 - val_loss: 0.8654 - val_accuracy: 0.6761
Epoch 3/5
1029/1029 [==============================] - 40s 39ms/step - loss: 0.5503 - accuracy: 0.8090 - val_loss: 0.9381 - val_accuracy: 0.6673
Epoch 4/5
1029/1029 [==============================] - 40s 39ms/step - loss: 0.4308 - accuracy: 0.8576 - val_loss: 1.0113 - val_accuracy: 0.6586
Epoch 5/5
1029/1029 [==============================] - 41s 40ms/step - loss: 0.3199 - accuracy: 0.8955 - val_loss: 1.2030 - val_accuracy: 0.6373


In [ ]:
model_2_results = calc_results_from_model(model_2)

In [ ]:
model_2_results

{'accuracy': 63.72691933916423,
 'f1': 0.6358959189951701,
 'precision': 0.637226273314714,
 'recall': 0.6372691933916423}

## GRU model gated recurrent unit 

In [ ]:
def compile_and_fit(model,name):
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
  model_history = model.fit(train_sent, train_labels, 
                              validation_data = (val_sent, val_labels), epochs =5,
                              callbacks = [create_tensorboard_callback(SAVE_DIR, name)])

In [ ]:
inputs = layers.Input(shape = (1, ), dtype = "string")
x = text_vectorizer(inputs)
x  =embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(5, activation="softmax")(x)
model_3 = tf.keras.Model(inputs, outputs)
model_3.summary()

In [ ]:
compile_and_fit(model_3, "model_3_GRU")

Epoch 1/5
1029/1029 [==============================] - 46s 42ms/step - loss: 0.6526 - accuracy: 0.7573 - val_loss: 1.0978 - val_accuracy: 0.6359
Epoch 2/5
1029/1029 [==============================] - 41s 39ms/step - loss: 0.3167 - accuracy: 0.8993 - val_loss: 1.2831 - val_accuracy: 0.6290
Epoch 3/5
1029/1029 [==============================] - 42s 41ms/step - loss: 0.2207 - accuracy: 0.9309 - val_loss: 1.4868 - val_accuracy: 0.6248
Epoch 4/5
1029/1029 [==============================] - 41s 40ms/step - loss: 0.1622 - accuracy: 0.9508 - val_loss: 1.7147 - val_accuracy: 0.6092
Epoch 5/5
1029/1029 [==============================] - 41s 39ms/step - loss: 0.1229 - accuracy: 0.9630 - val_loss: 1.9212 - val_accuracy: 0.6048


In [ ]:
model_3_results = calc_results_from_model(model_3)
model_3_results

{'accuracy': 60.48347910592808,
 'f1': 0.6049357966536043,
 'precision': 0.6077361572433394,
 'recall': 0.6048347910592808}

## Bidirectional RNN model

In [ ]:
inputs = layers.Input(shape =(1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(5, activation ="softmax")(x)
model_4 = tf.keras.Model(inputs, outputs)
model_4.summary()

In [ ]:
compile_and_fit(model_4, "model_4_BidirectionalLSTM")

Epoch 1/5
1029/1029 [==============================] - 69s 64ms/step - loss: 0.5846 - accuracy: 0.7794 - val_loss: 1.2507 - val_accuracy: 0.6104
Epoch 2/5
1029/1029 [==============================] - 63s 61ms/step - loss: 0.2116 - accuracy: 0.9326 - val_loss: 1.5969 - val_accuracy: 0.6097
Epoch 3/5
1029/1029 [==============================] - 62s 60ms/step - loss: 0.1250 - accuracy: 0.9629 - val_loss: 1.8969 - val_accuracy: 0.6031
Epoch 4/5
1029/1029 [==============================] - 64s 63ms/step - loss: 0.0835 - accuracy: 0.9752 - val_loss: 2.1575 - val_accuracy: 0.5982
Epoch 5/5
1029/1029 [==============================] - 62s 60ms/step - loss: 0.0610 - accuracy: 0.9824 - val_loss: 2.3418 - val_accuracy: 0.5929


In [ ]:
model_4_results = calc_results_from_model(model_4)
model_4_results

{'accuracy': 59.2930029154519,
 'f1': 0.5941155065507838,
 'precision': 0.5961544158007275,
 'recall': 0.592930029154519}

In [ ]:
inputs = layers.Input(shape =(1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.GRU(64))(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(5, activation ="softmax")(x)
model_5 = tf.keras.Model(inputs, outputs)
model_5.summary()

In [ ]:
compile_and_fit(model_5, "model_5_BidirectionalGRU")

Epoch 1/5
1029/1029 [==============================] - 63s 58ms/step - loss: 0.4523 - accuracy: 0.8314 - val_loss: 1.7731 - val_accuracy: 0.5826
Epoch 2/5
1029/1029 [==============================] - 58s 57ms/step - loss: 0.0931 - accuracy: 0.9716 - val_loss: 2.3230 - val_accuracy: 0.5869
Epoch 3/5
1029/1029 [==============================] - 59s 57ms/step - loss: 0.0512 - accuracy: 0.9850 - val_loss: 2.6467 - val_accuracy: 0.5730
Epoch 4/5
1029/1029 [==============================] - 59s 58ms/step - loss: 0.0397 - accuracy: 0.9879 - val_loss: 2.8376 - val_accuracy: 0.5869
Epoch 5/5
1029/1029 [==============================] - 59s 57ms/step - loss: 0.0346 - accuracy: 0.9897 - val_loss: 2.9684 - val_accuracy: 0.5856


In [ ]:
model_5_results = calc_results_from_model(model_5)
model_5_results

{'accuracy': 58.56413994169096,
 'f1': 0.586135249218072,
 'precision': 0.5869308502454179,
 'recall': 0.5856413994169096}

## 1D Conv Model Temporal Convolution 

In [ ]:
inputs = layers.Input(shape = (1,), dtype = "string")
x =text_vectorizer(inputs)
x =embedding(x)
x = layers.Conv1D(32, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(5, activation = 'softmax')(x)
model_6 = tf.keras.Model(inputs, outputs)
model_6.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_9 (TextVe (None, 31)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 31, 128)           1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 27, 32)            20512     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 32)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 5)                 165       
Total params: 1,300,677
Trainable params: 1,300,677
Non-trainable params: 0
________________________________________________

In [ ]:
compile_and_fit(model_6, "model_6_Temporal_conv")

Epoch 1/5
1029/1029 [==============================] - 24s 23ms/step - loss: 0.7369 - accuracy: 0.7322 - val_loss: 1.1932 - val_accuracy: 0.5801
Epoch 2/5
1029/1029 [==============================] - 23s 23ms/step - loss: 0.4488 - accuracy: 0.8492 - val_loss: 1.4433 - val_accuracy: 0.5544
Epoch 3/5
1029/1029 [==============================] - 22s 22ms/step - loss: 0.2913 - accuracy: 0.9091 - val_loss: 1.6656 - val_accuracy: 0.5627
Epoch 4/5
1029/1029 [==============================] - 23s 23ms/step - loss: 0.1723 - accuracy: 0.9548 - val_loss: 1.9820 - val_accuracy: 0.5482
Epoch 5/5
1029/1029 [==============================] - 23s 22ms/step - loss: 0.0901 - accuracy: 0.9814 - val_loss: 2.3581 - val_accuracy: 0.5485


In [ ]:
model_6_results = calc_results_from_model(model_6)

In [ ]:
model_6_results

{'accuracy': 54.8469387755102,
 'f1': 0.5506072676009579,
 'precision': 0.5551906492521282,
 'recall': 0.548469387755102}

## Transfer Learning USE

In [ ]:
import tensorflow_hub as hub

In [ ]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape =[],
                                        dtype = tf.string,
                                        trainable= False,
                                        name = "use")

In [ ]:
model_7 = tf.keras.Sequential([
                               #layers.Input(shape = (1,), dtype = "string"),
                               sentence_encoder_layer,
                               #layers.Dense(1024, activation= "relu"),
                               layers.Dense(64, activation= "relu"),
                               layers.Dense(5, activation="softmax")
])
compile_and_fit(model_7, "model_7_USE")

Epoch 1/5
1029/1029 [==============================] - 83s 24ms/step - loss: 1.2552 - accuracy: 0.4539 - val_loss: 1.1897 - val_accuracy: 0.4821
Epoch 2/5
1029/1029 [==============================] - 30s 29ms/step - loss: 1.1635 - accuracy: 0.5019 - val_loss: 1.1730 - val_accuracy: 0.4911
Epoch 3/5
1029/1029 [==============================] - 24s 23ms/step - loss: 1.0800 - accuracy: 0.5446 - val_loss: 1.1762 - val_accuracy: 0.4982
Epoch 4/5
1029/1029 [==============================] - 22s 21ms/step - loss: 0.9615 - accuracy: 0.6072 - val_loss: 1.2293 - val_accuracy: 0.4949
Epoch 5/5
1029/1029 [==============================] - 22s 21ms/step - loss: 0.8102 - accuracy: 0.6797 - val_loss: 1.3089 - val_accuracy: 0.4776


In [ ]:
model_7_results = calc_results_from_model(model_7)
model_7_results

{'accuracy': 47.76482021379981,
 'f1': 0.4796995799010443,
 'precision': 0.4847662596975694,
 'recall': 0.4776482021379981}

In [ ]:
val_df = pd.DataFrame({"text": val_sent,
                       "target": val_labels,
                       "pred": model_1_preds,
                       "pred_prob": tf.squeeze(model_1_predprobs)})
val_df.head()

,text,target,pred,pred_prob
0,This is the latest corona challenge.Anyone w...,2,0,"(tf.Tensor(0.58511055, shape=(), dtype=float32..."
1,"Cheers buddy! Just had lunch at 2.50am, as wor...",1,1,"(tf.Tensor(0.000523297, shape=(), dtype=float3..."
2,Mask = 2020 Essential?\r\r\nWashable Mask = Re...,2,2,"(tf.Tensor(0.05821156, shape=(), dtype=float32..."
3,Although nobody truly knows what the world wil...,1,1,"(tf.Tensor(2.669141e-05, shape=(), dtype=float..."
4,Insane despicable profiting from misery The ma...,2,2,"(tf.Tensor(0.03669692, shape=(), dtype=float32..."


In [ ]:
pred_class = tf.argmax(model_1.predict(["#Beirut declared a “devastated city”, two-week state of emergency officially declared. #Lebanon"]), axis =1)

In [ ]:
pred_class

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>

## One Hot encoded Trials

In [ ]:
train_df_shuffled.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
31089,34888,79840,Lagos,06-04-2020,Without the there would not be any problem wh...,Neutral
35564,39363,84315,NaN,09-04-2020,Rice &amp; wheat prices surge amid fears Covid...,Extremely Negative
144,3943,48895,NaN,16-03-2020,When the government says to start social dista...,Positive
8202,12001,56953,irlande du nord,19-03-2020,What the shops are doing is obeying the law of...,Positive
31720,35519,80471,"Zaria, Nigeria",07-04-2020,Kaduna State Task Force on Covid 19 led by the...,Negative


In [ ]:
train_df_onehot_labels = pd.get_dummies(train_df_shuffled["Sentiment"])
train_df_onehot_labels.head()

,Extremely Negative,Extremely Positive,Negative,Neutral,Positive
31089,0,0,0,1,0
35564,1,0,0,0,0
144,0,0,0,0,1
8202,0,0,0,0,1
31720,0,0,1,0,0


In [ ]:
train_df_onehot_labels.to_numpy()

array([[0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       ...,
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0]], dtype=uint8)

In [ ]:
train_sent, val_sent, train_labels_oh, val_labels_oh =train_test_split(train_df_shuffled["OriginalTweet"].to_numpy(), 
                                                                 train_df_onehot_labels.to_numpy(), 
                                                                 test_size = 0.2, 
                                                                 random_state = 42)

In [ ]:
model_1_oh = tf.keras.models.clone_model(model_1)

In [ ]:
model_1_oh.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_1_oh.fit(train_sent, train_labels_oh, 
            validation_data = (val_sent, val_labels_oh), 
            epochs = 5, 
            callbacks = [create_tensorboard_callback(SAVE_DIR, "model_1_oh_dense")])

Epoch 1/5


FailedPreconditionError: ignored